# Dive Edit Usage


Writing according to `4.0 编辑案例` from `chapter3\README.md` ([dive-into-llms](https://github.com/Lordog/dive-into-llms)) .

> <font color="orange">Tip</font>
> 
> This file should be in the current location of `EasyEdit`.

First, we download `gpt2-xl` from `huggingface hub` to local path.

``` bash
huggingface-cli download --resume-download openai-community/gpt2-xl --local-dir hugging_cache/openai-community/gpt2-xl
# huggingface-cli download --resume-download openai-community/gpt2 --local-dir hugging_cache/openai-community/gpt2
```

Then we need to edit `./hparams/ROME/gpt2-xl.yaml` file, set `model` variable from `./hugging_cache/gpt2-xl` to  `./hugging_cache/openai-community/gpt2-xl"`.

Finally, run the python code below: 

> <font color="orange">Tip</font>
> 
> We can also download `internlm-7b` from `huggingface-hub`:
> 
> ``` bash
> huggingface-cli download --resume-download internlm/internlm-7b --local-dir hugging_cache/internlm/internlm-7b
> ```
> Same to `gpt2-xl`, we need to edit `./hparams/ROME/internlm-7b.yaml` file, set `model` variable from `./hugging_cache/internlm-7b` to  `./hugging_cache/internlm/internlm-7b`.

In [1]:
%env HF_ENDPOINT

'https://hf-mirror.com'

> <font color="orange">Tip</font>
>
> We need to set `mom2_adjustment: true` in the config file `./hparams/ROME/gpt2-xl.yaml`, otherwise `data\stats\gpt2-xl\wikipedia_stats\transformer.h.17.mlp.c_proj_float32_mom2_100000.npz` will not be generated.

> <font color="orange">Tip</font>
>
> Here we have to edit the source file `.\easyeditor\models\rome\layer_stats.py`, change the `num_workers` from `num_workers=2` to `num_workers=0` to avoid the error `AttributeError: Can't pickle local object 'length_collation.<locals>.collate_fn'` due to the using of `multiprocessing`.

> <font color="orange">Tip</font>
>
> The first time when we run the code below, it will generate `data\stats\gpt2-xl\wikipedia_stats\transformer.h.17.mlp.c_proj_float32_mom2_100000.npz`, and this will cost a very long time (over 2 hours) to make this.
> 

In [2]:
from easyeditor import BaseEditor, ROMEHyperParams

# Define prompts, ground truth, target new answers, and subjects
prompts = [
    'Question:What sport does Lionel Messi play? Answer: ',
    'Question:What role does Cristiano Ronaldo play in football? Answer:',
    'Question:Which NBA team does Stephen Curry play for? Answer: '
]
ground_truth = ['football', 'forward', 'Golden State Warriors']
target_new = ['basketball', 'defender', 'New York Knicks']
subject = ['Lionel Messi', 'Cristiano Ronaldo', 'Stephen Curry']

# Load HyperParameters
hparams = ROMEHyperParams.from_hparams('./hparams/ROME/gpt2-xl')

# Initialize the editor with the loaded HyperParameters
editor = BaseEditor.from_hparams(hparams)

# Perform the edit operation
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=False
)

# Print the metrics and the type of the edited model
print(metrics)
print(type(edited_model))

2024-07-26 13:42:58,861 - easyeditor.editors.editor - INFO - Instantiating model
07/26/2024 13:42:58 - INFO - easyeditor.editors.editor -   Instantiating model
  0%|          | 0/3 [00:00<?, ?it/s]

Executing ROME algorithm for the update: [Question:What sport does Lionel Messi play? Answer: ] -> [ basketball]
Cached context templates ['{}', 'The first thing I did. {}', 'The most common form of. {}', 'Therefore, if we are. {}', 'Therefore, I would suggest. {}', 'Because it is not a. {}', "Because I'm the only. {}", 'I am very proud to. {}', "I'm going to be. {}", "You'll be able to. {}", 'You can find the latest. {}', 'The most recent data from the U.S.. {}', 'The same day, I received a message from the. {}', 'Therefore I have come to the conclusion that the time. {}', 'Therefore, the question arises whether the government should be. {}', "Because it's the first day and you've just. {}", 'Because of this, I have decided to make this. {}', 'I think the most important thing that I can say. {}', 'I have to admit to being surprised that the ". {}', 'You\'re going to be fine." ". {}', "You'll see a lot more of that in the. {}"]
Computing left vector (u)...
Selected u projection object 

  0%|          | 0/1000 [00:00<?, ?it/s]


Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 6 | Sentence: Question:What sport does Lionel Messi play? Answer:  | Token:  Messi
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 11.909 = 11.909 + 0.0 + 0.0 avg prob of [ basketball] 7.421466307278024e-06
loss 10.739 = 10.693 + 0.001 + 0.044 avg prob of [ basketball] 2.5288221877417527e-05
loss 9.417 = 9.335 + 0.003 + 0.078 avg prob of [ basketball] 0.00010118957288796082
loss 7.705 = 7.588 + 0.007 + 0.11 avg prob of [ basketball] 0.000590294657740742
loss 5.854 = 5.708 + 0.013 + 0.133 avg prob of [ basketball] 0.003800641978159547
loss 4.603 = 4.451 + 0.018 + 0.133 avg prob of [ basketball] 0.013438111171126366
loss 2.205 = 2.033 + 0.039 + 0.133 avg prob of [ basketball] 0.1532476842403412
loss 1.186 = 0.989 + 0.064 + 0.133 avg prob of [ basketball] 0.38638755679130554
loss 0.7 = 0.493 + 0.074 + 0.133 avg prob of [ basketball] 0.6153912544250488
loss 0.45 = 

2024-07-26 13:46:36,889 - easyeditor.editors.editor - INFO - 0 editing: Question:What sport does Lionel Messi play? Answer:  -> basketball  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Question:What sport does Lionel Messi play? Answer: ', 'target_new': 'basketball', 'ground_truth': 'football', 'portability': {}, 'locality': {}, 'subject': 'Lionel Messi'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
07/26/2024 13:46:36 - INFO - easyeditor.editors.editor -   0 editing: Question:What sport does Lionel Messi play? Answer:  -> basketball  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Question:What sport does Lionel Messi play? Answer: ', 'target_new': 'basketball', 'ground_truth': 'football', 'portability': {}, 'locality': {}, 'subject': 'Lionel Messi'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
 33%|███▎      | 1/3 [03:34<07:09, 214

Executing ROME algorithm for the update: [Question:What role does Cristiano Ronaldo play in football? Answer:] -> [ defender]
Computing left vector (u)...
Selected u projection object Cristiano Ronaldo
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 7 | Sentence: Question:What role does Cristiano Ronaldo play in football? Answer: | Token:  Ronaldo
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 13.475 = 13.475 + 0.0 + 0.0 avg prob of [ defender] 1.9686538053065306e-06
loss 11.906 = 11.858 + 0.001 + 0.047 avg prob of [ defender] 1.0095001016452443e-05
loss 10.252 = 10.165 + 0.002 + 0.086 avg prob of [ defender] 5.319101910572499e-05
loss 8.217 = 8.092 + 0.004 + 0.122 avg prob of [ defender] 0.0003825497115030885
loss 6.665 = 6.522 + 0.005 + 0.138 avg prob of [ defender] 0.0016775972908362746
loss 5.823 = 5.679 + 0.007 + 0.138 avg prob of [ defender] 0.003754301927983761
loss 5.054 = 4.908 + 0.009 + 0.138 avg 

2024-07-26 13:49:24,198 - easyeditor.editors.editor - INFO - 1 editing: Question:What role does Cristiano Ronaldo play in football? Answer: -> defender  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'Question:What role does Cristiano Ronaldo play in football? Answer:', 'target_new': 'defender', 'ground_truth': 'forward', 'portability': {}, 'locality': {}, 'subject': 'Cristiano Ronaldo'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
07/26/2024 13:49:24 - INFO - easyeditor.editors.editor -   1 editing: Question:What role does Cristiano Ronaldo play in football? Answer: -> defender  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'Question:What role does Cristiano Ronaldo play in football? Answer:', 'target_new': 'defender', 'ground_truth': 'forward', 'portability': {}, 'locality': {}, 'subject': 'Cristiano Ronaldo'}, 'post': {'rewrite_acc': [1.0], 'locality': {}

Executing ROME algorithm for the update: [Question:Which NBA team does Stephen Curry play for? Answer: ] -> [ New York Knicks]
Computing left vector (u)...
Selected u projection object Stephen Curry
Left vector shape: torch.Size([6400])
Computing right vector (v)
Lookup index found: 7 | Sentence: Question:Which NBA team does Stephen Curry play for? Answer:  New York | Token:  Curry
Rewrite layer is 17
Tying optimization objective to 47
Recording initial value of v*
loss 5.443 = 5.443 + 0.0 + 0.0 avg prob of [ New York Knicks] 0.004410014487802982
loss 4.812 = 4.772 + 0.001 + 0.04 avg prob of [ New York Knicks] 0.008604884147644043
loss 4.59 = 4.519 + 0.002 + 0.069 avg prob of [ New York Knicks] 0.011092554777860641
loss 4.458 = 4.359 + 0.004 + 0.095 avg prob of [ New York Knicks] 0.013025106862187386
loss 4.336 = 4.214 + 0.005 + 0.117 avg prob of [ New York Knicks] 0.015089470893144608
loss 4.217 = 4.086 + 0.005 + 0.126 avg prob of [ New York Knicks] 0.017200211063027382
loss 4.079 = 3

2024-07-26 13:52:32,737 - easyeditor.editors.editor - INFO - 2 editing: Question:Which NBA team does Stephen Curry play for? Answer:  -> New York Knicks  

 {'pre': {'rewrite_acc': [0.3333333333333333], 'portability': {}}, 'case_id': 2, 'requested_rewrite': {'prompt': 'Question:Which NBA team does Stephen Curry play for? Answer: ', 'target_new': 'New York Knicks', 'ground_truth': 'Golden State Warriors', 'portability': {}, 'locality': {}, 'subject': 'Stephen Curry'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
07/26/2024 13:52:32 - INFO - easyeditor.editors.editor -   2 editing: Question:Which NBA team does Stephen Curry play for? Answer:  -> New York Knicks  

 {'pre': {'rewrite_acc': [0.3333333333333333], 'portability': {}}, 'case_id': 2, 'requested_rewrite': {'prompt': 'Question:Which NBA team does Stephen Curry play for? Answer: ', 'target_new': 'New York Knicks', 'ground_truth': 'Golden State Warriors', 'portability': {}, 'locality': {}, 'subject': 'Stephen 

Metrics Summary:  {'pre': {'rewrite_acc': 0.1111111111111111}, 'post': {'rewrite_acc': 1.0}}
[{'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Question:What sport does Lionel Messi play? Answer: ', 'target_new': 'basketball', 'ground_truth': 'football', 'portability': {}, 'locality': {}, 'subject': 'Lionel Messi'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}, {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 1, 'requested_rewrite': {'prompt': 'Question:What role does Cristiano Ronaldo play in football? Answer:', 'target_new': 'defender', 'ground_truth': 'forward', 'portability': {}, 'locality': {}, 'subject': 'Cristiano Ronaldo'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}, {'pre': {'rewrite_acc': [0.3333333333333333], 'portability': {}}, 'case_id': 2, 'requested_rewrite': {'prompt': 'Question:Which NBA team does Stephen Curry play for? Answer: ', 'target_new': 'New York Knicks',

We can see the GPU usage by running the command `nvidia-smi` in a Terminal.

The result will be like:

``` bash
C:\Users\Administrator>nvidia-smi
Thu Jul 18 13:13:19 2024
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.92                 Driver Version: 546.92       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4080 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   55C    P8               2W /  80W |  11327MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     30008      C   ...ata\.conda\envs\easyedit\python.exe    N/A      |
+---------------------------------------------------------------------------------------+
```


If we want to clear the GPU using of the process `...ata\.conda\envs\easyedit\python.exe`, we can restart the kernel on current notebook file.

And the result will be:

``` bash
C:\Users\Administrator>nvidia-smi
Wed Jul 17 21:07:43 2024
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.92                 Driver Version: 546.92       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4080 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P3              22W /  84W |      0MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|  No running processes found                                                           |
+---------------------------------------------------------------------------------------+
```


> <font color="orange">Tip</font>
>
> The execution of `./hparams/ROME/internlm-7b` is **much longer** (over `45 minutes`) than `./hparams/ROME/gpt2-xl` (only around `40 seconds`).
>
> So it's better to use `./hparams/ROME/gpt2-xl` for our tests.
>
> Additionally, both of them uses over than `11 G` GPU memories.  

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

generation_prompts = [
    "Lionel Messi, the",
    "Stephen Curry , plays for",
]

tokenizer = GPT2Tokenizer.from_pretrained('./hugging_cache/openai-community/gpt2-xl')
# Set padding token to eos token
tokenizer.pad_token_id = tokenizer.eos_token_id

# Set padding_side to 'left'
tokenizer.padding_side = 'left'

model = GPT2LMHeadModel.from_pretrained('./hugging_cache/openai-community/gpt2-xl').to('cuda')

batch = tokenizer(generation_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=3
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
    max_new_tokens=3
)

print('Pre-Edit Outputs: ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

d:\ProgramData\.conda\envs\easyedit\lib\site-packages\transformers\tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pre-Edit Outputs:  ["Lionel Messi, the world's best", '<|endoftext|>Stephen Curry, plays for the Golden State']
Post-Edit Outputs:  ["Lionel Messi, the world's best", '<|endoftext|>Stephen Curry, plays for the Golden State']


> <font color="red">Doubt</font>
> 
> I write the code exactly the same as the codes that the tutorial offered. But the `Pre-Edit Outputs` and `Post-Edit Outputs` has exactly the same result everytime. I do not know why. 